## Mitsuba Data Generation

Generates noisy/clean pairs from Benedikt Bitterli Rendering Resources (https://benedikt-bitterli.me/resources/)

In [1]:
import mitsuba as mi
import zipfile
import tempfile
import os
import numpy as np
from PIL import Image

mi.set_variant('scalar_rgb')

In [2]:
def load_scene_from_folder(folder_path):
    folder_to_search = folder_path  # Use original folder directly

    # Try to find the XML scene file
    xml_files = [f for f in os.listdir(folder_to_search) if f.endswith('.xml')]
    if not xml_files:
        raise FileNotFoundError("No .xml scene file found in folder.")
    
    scene_path = os.path.join(folder_to_search, xml_files[0])
    scene = mi.load_file(scene_path)
    
    return scene, folder_to_search

In [3]:
def render_photon_tensor(scene, num_frames=3, spp_per_frame=1):
    sensor = scene.sensors()[0]
    film = sensor.film()
    res = film.crop_size()
    print(f"Rendering resolution: {res}")

    T, H, W = num_frames, res[1], res[0]
    photon_tensor = np.zeros((T, H, W, 3), dtype=np.float32)

    for t in range(num_frames):
        print(f"Rendering frame {t + 1}/{num_frames}")

        # Set a new random seed for stochastic samples
        sensor.sampler().seed(t)

        film.clear()
        mi.render(scene, sensor=sensor)
        bmp = film.bitmap(raw=True).convert(mi.Bitmap.PixelFormat.RGB, mi.Struct.Type.Float32)
        img_np = np.array(bmp)
        photon_tensor[t] = img_np
    
    return photon_tensor


In [4]:
def save_photon_tensor(tensor, out_dir="output_frames", prefix="frame"):
    os.makedirs(out_dir, exist_ok=True)
    for t in range(tensor.shape[0]):
        img = (np.clip(tensor[t], 0, 1) * 255).astype(np.uint8)
        Image.fromarray(img).save(os.path.join(out_dir, f"{prefix}_{t:02d}.png"))
    np.save(os.path.join(out_dir, "photon_tensor.npy"), tensor)
    print(f"Saved photon tensor and {tensor.shape[0]} frames.")

In [5]:
zip_file_path = "spaceship"

scene, temp_dir = load_scene_from_folder(zip_file_path)
photon_tensor = render_photon_tensor(scene, num_frames=1, spp_per_frame=1)
save_photon_tensor(photon_tensor, out_dir="spaceship-output")

Rendering resolution: [1280, 720]
Rendering frame 1/1
Saved photon tensor and 1 frames.


In [6]:
data = np.load("spaceship-output/photon_tensor.npy")

print("Shape:", data.shape)
print("Data type:", data.dtype)

print("\nSample values:")
print(data[:, 5, 5, :])

Shape: (1, 720, 1280, 3)
Data type: float32

Sample values:
[[0.01842426 0.01842426 0.0184283 ]]


In [7]:
zip_file_path = "water"

scene, temp_dir = load_scene_from_folder(zip_file_path)
photon_tensor = render_photon_tensor(scene, num_frames=1, spp_per_frame=1)
save_photon_tensor(photon_tensor, out_dir="water-output")

Rendering resolution: [1280, 720]
Rendering frame 1/1
Saved photon tensor and 1 frames.


In [8]:
data = np.load("water-output/photon_tensor.npy")

print("Shape:", data.shape)
print("Data type:", data.dtype)

print("\nSample values:")
print(data[:, 5, 5, :])

Shape: (1, 720, 1280, 3)
Data type: float32

Sample values:
[[0.01534068 0.02325414 0.02874304]]
